<a href="https://colab.research.google.com/github/mining-software-repositories/treinamento/blob/main/test_cassandra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Clona o repositório Cassandra
!git clone https://github.com/apache/cassandra.git

In [ ]:
# Clona o repositorio msr_test para acessar as libs proprietárias
!git clone https://github.com/giselesousar/msr_tests

In [ ]:
#Mostra todos os commits entre as tags 3.0.0 e 3.11.11
!cd cassandra && git log --pretty="%H;%ai;%s" cassandra-3.0.0...cassandra-3.11.11 > commits-3.0.0-3.11.11-full.txt
!cd cassandra && git log --pretty="%H %s" cassandra-3.0.0...cassandra-3.11.11 > commits-3.0.0-3.11.11.txt
!echo "10 primeiros commits entre as versões cassandra-3.0.0...cassandra-3.11.11"
!cd cassandra && head commits-3.0.0-3.11.11.txt
!echo "..."

# Total de commits registrados entre as versões cassandra-3.0.0...cassandra-3.11.11
!echo "Total de commits registrados entre as versões cassandra-3.0.0...cassandra-3.11.11: "
!cd cassandra && cat commits-3.0.0-3.11.11.txt | wc -l

In [ ]:
# Consulta Commits e arquivos
# Lista os commits entre duas tags
!cd cassandra && git log --pretty="%H" cassandra-3.0.0...cassandra-3.11.11 > commmitstags300to31111.txt
commits_tag_3_from_000_to_111111 = !cd cassandra && cat commmitstags300to31111.txt
print(f'Qtd: { len(commits_tag_3_from_000_to_111111) }, {commits_tag_3_from_000_to_111111}')

In [ ]:
# Baixa o arquivo sqlite do banco dos commmits do Cassandra
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1n2RE3xsLtD-fv_omcI6vtm6x5si-PxmZ' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1n2RE3xsLtD-fv_omcI6vtm6x5si-PxmZ" -O msrcassandra300to31111.db && rm -rf /tmp/cookies.txt

In [ ]:
!pip install pydriller

In [ ]:
import treinamento.utils.dao as dao
import treinamento.utils.utility as utility
import datetime
from pydriller import Repository

path_repository = 'cassandra'

def load_db(path_repository, create=False):
    ## 1. Carrega o banco de dados e cria as estruturas das tabelas
    print(f'Configura e carrega o banco {dao.data_base}...')
    db_session = dao.create_session()
    print(f'Banco {dao.data_base} carregado com sucesso! \n')

    # 2. Carrega os manipuladores de CommitComplete e FileComplete
    commitsCompleteCollection = dao.CommitsCompleteCollection(session=db_session)
    filesCompleteCollection = dao.FilesCompleteCollection(session = db_session)

    if create: 
        dao.drop_tables()
        dao.create_tables()

        # 3. É preciso percorrer todos os commits, seus arquivos modificados e salva-los no Banco
        print('Percorrendo todos os commits da lista commits_tag_3_from_000_to_111111...')
        t1 = datetime.datetime.now()
        print(t1)

        for commit in Repository(path_repository, only_commits=commits_tag_3_from_000_to_111111).traverse_commits():    
            c = dao.CommitComplete(name = commit.hash, 
                hash = commit.hash, 
                msg = commit.msg,
                author = utility.concat_str(commit.author.name,commit.author.email), 
                committer = utility.concat_str(commit.committer.name,commit.committer.email), 
                author_date = commit.author_date,
                author_timezone = commit.author_timezone,
                committer_date = commit.committer_date,
                committer_timezone = commit.committer_timezone,
                branches = utility.convert_list_to_str(commit.branches),
                in_main_branch = commit.in_main_branch,
                merge = commit.merge,
                modified_files = utility.convert_modifield_list_to_str(commit.modified_files),
                parents = utility.convert_list_to_str(commit.parents),
                project_name = commit.project_name,
                project_path = commit.project_path,
                deletions = commit.deletions,
                insertions = commit.insertions,
                lines = commit.lines,
                files = commit.files,
                dmm_unit_size = commit.dmm_unit_size,
                dmm_unit_complexity = commit.dmm_unit_complexity,
                dmm_unit_interfacing = commit.dmm_unit_interfacing)
            
            commitsCompleteCollection.insert_commit(c)

            for m in commit.modified_files:
                commit_by_hash = commitsCompleteCollection.query_commit_by_hash(commit.hash)
                if m is not None and  m.filename is not None:
                    if '.java' in m.filename:
                        is_java = True
                    else:
                        is_java = False
                    mf = dao.FileComplete(
                        name = m.filename,
                        hash = commit.hash,
                        is_java = is_java,
                        old_path = m.old_path,
                        new_path = m.new_path,
                        filename = m.filename,
                        change_type = m.change_type.name,
                        diff = str(m.diff),
                        diff_parsed = utility.convert_dictionary_to_str(m.diff_parsed),
                        added_lines = m.added_lines,
                        deleted_lines = m.deleted_lines,
                        source_code = str(m.source_code),
                        source_code_before = str(m.source_code_before),
                        methods = utility.convert_list_to_str(m.methods),
                        methods_before = utility.convert_list_to_str(m.methods_before),
                        changed_methods = utility.convert_list_to_str(m.changed_methods),
                        nloc = m.nloc,
                        complexity = m.complexity,
                        token_count = m.token_count, 
                        commit_id = commit_by_hash.id
                    )
                    # salva o arquivo correte
                    filesCompleteCollection.insert_file(mf)

        t2 = datetime.datetime.now()
        print(t2)
        print(f'Analise concluida em: {t2 -t1}')

    return filesCompleteCollection, commitsCompleteCollection

In [ ]:
# 1. Carrega o banco de dados
# Obs: o parâmetro create deve ser setado para True caso deseje recriar o banco do zero.
filesCompleteCollection, commitsCompleteCollection = load_db(path_repository, create=False)

In [ ]:
# Carrega as tabelas do banco em dataframes

import pandas as pd
import sqlite3

DATA_BASE='msrcassandra300to31111.db'
con = sqlite3.connect(DATA_BASE)

my_query_commits = "select * from commitscomplete"
my_query_files = "select * from filescomplete"
my_query_files_commits = "select f.id as 'file_id', f.hash as 'file_hash_commit', f.description as 'file_description', f.is_java as 'file_is_java', f.created_date as 'file_created_date', f.old_path as 'file_old_path', f.new_path as 'file_new_path', f.filename as 'file_filename', f.change_type as 'file_change_type', f.diff as 'file_diff', f.diff_parsed as 'file_diff_parsed', f.added_lines as 'file_added_lines', f.deleted_lines as 'file_deleted_lines', f.source_code as 'file_source_code', f.source_code_before as 'file_source_code_before', f.nloc as 'file_nloc', f.complexity as 'file_complexity', f.token_count as 'file_token_count', f.commit_id as 'file_commit_id', c.* from filescomplete f, commitscomplete c where f.commit_id=c.id"

df_commits_from_db = pd.read_sql_query(my_query_commits, con)
df_files_from_db = pd.read_sql_query(my_query_files, con)
df_files_commits_from_db = pd.read_sql(my_query_files_commits, con)

con.close() 

In [ ]:
# Faz alguns ajustes nos dataframes
df_files_from_db['modified_lines'] = df_files_from_db.added_lines + df_files_from_db.deleted_lines
df_files_commits_from_db['modified_lines'] = df_files_commits_from_db.file_added_lines + df_files_commits_from_db.file_deleted_lines

In [ ]:
# Lista os commits e seus arquivos modificados
df_commits_from_db[['name', 'modified_files']]

In [ ]:
# procura por um commit especifico
df_commits_from_db[['name', 'modified_files']].query("name == '47341eb6aaca318d0ffc0e9f906b98db50b9e9ff'")

In [ ]:
# Lista todos os arquivos e seus commits
df_files_from_db[['name', 'hash']].sort_values('name')

In [ ]:
# Mostra as Complexidades Ciclomáticas dos arquivos
df_files_commits_from_db[['file_filename', 'file_complexity', 'author_date']].sort_values(by=['file_filename', 'author_date'], ascending=True)

In [ ]:
# Mostra as complexidades ciclomáticas de um determinado arquivo
df_files_commits_from_db[['file_filename', 'file_complexity', 'author_date']].sort_values(by=['file_filename', 'author_date'], ascending=True).query("file_filename == 'StorageService.java'")

In [ ]:
# calcula frequência dos arquivos na faixa de commits analisados
list_of_files_frequency_in_commits = {}

# Dataframe agrupados por arquivos e seus commits
df_groupby_name = df_files_from_db[['name', 'hash']].groupby('name')

print(f'Quantidade de grupos: {df_groupby_name.ngroups}')
print(f'Grupos: {df_groupby_name.groups}')

In [ ]:
group_files = df_groupby_name.size()
print(group_files)

In [ ]:
list_of_files_frequency_in_commits = group_files.to_dict()
print(f'{ len(list_of_files_frequency_in_commits) }, {list_of_files_frequency_in_commits}')

In [ ]:
df_files_from_db[['name','modified_lines']]

In [ ]:
df_groupby_name_modified_lines = df_files_from_db[['name','modified_lines']].groupby('name')

group_files_modified_lines = df_groupby_name_modified_lines.sum()

list_of_files_modified_lines = group_files_modified_lines.to_dict()
print(f'{ len(list_of_files_modified_lines) }, {list_of_files_modified_lines}')